In [10]:



#Assigning Dice that implements method roll that rolls two 6-sided dice
from numpy import random as r #importing random from numpy to create random values for rolling the dice 
class Dice(object): # Assigning class Dice

    def __init__(self): #initializing the object’s state
        self.n_rolls = 0 #Initial number of rolls for the dice

    def roll(self): #implementing method roll that rolls two 6-sided dice
        self.n_rolls += 1 #enabling 1 roll per 1 call of function
        self.result = r.randint(1, 7, size=2) #creating random values for rolling two dice
        self.total = sum(self.result) #finding the sum of the results on the dice

if __name__ == "__main__": #module’s name is available as value to __name__ global variable

    d1 = Dice() #declaring Dice object
    d1.roll()
    d1.roll()
    d1.roll() #rolling the dice three times per calling the object
   

    



#Defining a Class for the player 
class Player(object):
    
    def __init__(self, bankroll, bet_strategy=None, name="Player"): 
        self.bankroll = bankroll
        self.bet_strategy = bet_strategy
        self.name = name
        self.bets_on_table = []
        self.total_bet_amount = 0
        # TODO: initial betting strategy

    def bet(self, bet_object):
        if self.bankroll >= bet_object.bet_amount:
            self.bankroll -= bet_object.bet_amount
            self.bets_on_table.append(
                bet_object
            )  # TODO: make sure this only happens if that bet isn't on the table, otherwise wager amount gets updated
            self.total_bet_amount += bet_object.bet_amount

    def remove(self, bet_object):
        # TODO: add bet attribute for whether a bet can be removed and put condition in here
        if bet_object in self.bets_on_table:
            self.bankroll += bet_object.bet_amount
            self.bets_on_table.remove(bet_object)
            self.total_bet_amount -= bet_object.bet_amount

    def has_bet(self, *bets_to_check):
        """ returns True if bets_to_check and self.bets_on_table has at least one thing in common """
        bet_names = {b.name for b in self.bets_on_table}
        return bool(bet_names.intersection(bets_to_check))

    def get_bet(self, bet_name, bet_subname=""):
        """returns first betting object matching bet_name and bet_subname.
        If bet_subname="Any", returns first betting object matching bet_name"""
        if bet_subname == "Any":
            bet_name_list = [b.name for b in self.bets_on_table]
            ind = bet_name_list.index(bet_name)
        else:
            bet_name_list = [[b.name, b.subname] for b in self.bets_on_table]
            ind = bet_name_list.index([bet_name, bet_subname])
        return self.bets_on_table[ind]

    def num_bet(self, *bets_to_check):
        """ returns the total number of bets in self.bets_on_table that match bets_to_check """
        bet_names = [b.name for b in self.bets_on_table]
        return sum([i in bets_to_check for i in bet_names])

    def remove_if_present(self, bet_name, bet_subname=""):
        if self.has_bet(bet_name):
            self.remove(self.get_bet(bet_name, bet_subname))

    def _add_strategy_bets(self, table, *args, **kwargs):
        """ Implement the given betting strategy """
        return self.bet_strategy(self, table, *args, **kwargs)

    def _update_bet(self, table_object, dice_object, verbose=False):
        info = {}
        for b in self.bets_on_table[:]:
            status, win_amount = b._update_bet(table_object, dice_object)

            if status == "win":
                self.bankroll += win_amount + b.bet_amount
                self.total_bet_amount -= b.bet_amount
                self.bets_on_table.remove(b)
                if verbose:
                    print(f"{self.name} won ${win_amount} on {b.name} bet!")
            elif status == "lose":
                self.total_bet_amount -= b.bet_amount
                self.bets_on_table.remove(b)
                if verbose:
                    print(f"{self.name} lost ${b.bet_amount} on {b.name} bet.")
            elif status == "push":
                self.bankroll += b.bet_amount
                self.total_bet_amount -= b.bet_amount
                self.bets_on_table.remove(b)
                if verbose:
                    print(f"{self.name} pushed ${b.bet_amount} on {b.name} bet.")

            info[b.name] = {"status": status, "win_amount": win_amount}
        return info






class Bet(object):
    """
    A generic bet for the craps table
    Parameters
    ----------
    bet_amount : float
        Wagered amount for the bet
    Attributes
    ----------
    name : string
        Name for the bet
    subname : string
        Subname, usually denotes number for a come/don't come bet
    winning_numbers : list
        Numbers to roll for this bet to win
    losing_numbers : list
        Numbers to roll that cause this bet to lose
    payoutratio : float
        Ratio that bet pays out on a win
    """

    name = None
    subname = ""
    winning_numbers = []
    losing_numbers = []
    payoutratio = float(1)
    # TODO: add whether bet can be removed

    def __init__(self, bet_amount):
        self.bet_amount = float(bet_amount)

    # def __eq__(self, other):
    #     return self.name == other.name

    def _update_bet(self, table_object, dice_object: Dice):
        status = None
        win_amount = 0

        if dice_object.total in self.winning_numbers:
            status = "win"
            win_amount = self.payoutratio * self.bet_amount
        elif dice_object.total in self.losing_numbers:
            status = "lose"

        return status, win_amount





class PassLine(Bet):
    # TODO: make this require that table_object.point = "Off",
    # probably better in the player module
    def __init__(self, bet_amount):
        self.name = "PassLine"
        self.winning_numbers = [7, 11]
        self.losing_numbers = [2, 3, 12]
        self.payoutratio = 1.0
        self.prepoint = True
        super().__init__(bet_amount)

    def _update_bet(self, table_object, dice_object):
        status = None
        win_amount = 0

        if dice_object.total in self.winning_numbers:
            status = "win"
            win_amount = self.payoutratio * self.bet_amount
        elif dice_object.total in self.losing_numbers:
            status = "lose"
        elif self.prepoint:
            self.winning_numbers = [dice_object.total]
            self.losing_numbers = [7]
            self.prepoint = False

        return status, win_amount


class Come(PassLine):
    def __init__(self, bet_amount):
        super().__init__(bet_amount)
        self.name = "Come"

    def _update_bet(self, table_object, dice_object):
        status, win_amount = super()._update_bet(table_object, dice_object)
        if not self.prepoint and self.subname == "":
            self.subname = "".join(str(e) for e in self.winning_numbers)
        return status, win_amount





class Odds(Bet):
    def __init__(self, bet_amount, bet_object):
        super().__init__(bet_amount)
        self.name = "Odds"
        self.subname = "".join(str(e) for e in bet_object.winning_numbers)
        self.winning_numbers = bet_object.winning_numbers
        self.losing_numbers = bet_object.losing_numbers

        if self.winning_numbers == [4] or self.winning_numbers == [10]:
            self.payoutratio = 2 / 1
        elif self.winning_numbers == [5] or self.winning_numbers == [9]:
            self.payoutratio = 3 / 2
        elif self.winning_numbers == [6] or self.winning_numbers == [8]:
            self.payoutratio = 6 / 5


"""
Place Bets on 4,5,6,8,9,10
"""


class Place(Bet):
    def _update_bet(self, table_object, dice_object):
        # place bets are inactive when point is "Off"
        if table_object.point == "On":
            return super()._update_bet(table_object, dice_object)
        else:
            return None, 0


class Place4(Place):
    def __init__(self, bet_amount):
        self.name = "Place4"
        self.winning_numbers = [4]
        self.losing_numbers = [7]
        self.payoutratio = 9 / 5
        super().__init__(bet_amount)


class Place5(Place):
    def __init__(self, bet_amount):
        self.name = "Place5"
        self.winning_numbers = [5]
        self.losing_numbers = [7]
        self.payoutratio = 7 / 5
        super().__init__(bet_amount)


class Place6(Place):
    def __init__(self, bet_amount):
        self.name = "Place6"
        self.winning_numbers = [6]
        self.losing_numbers = [7]
        self.payoutratio = 7 / 6
        super().__init__(bet_amount)


class Place8(Place):
    def __init__(self, bet_amount):
        self.name = "Place8"
        self.winning_numbers = [8]
        self.losing_numbers = [7]
        self.payoutratio = 7 / 6
        super().__init__(bet_amount)


class Place9(Place):
    def __init__(self, bet_amount):
        self.name = "Place9"
        self.winning_numbers = [9]
        self.losing_numbers = [7]
        self.payoutratio = 7 / 5
        super().__init__(bet_amount)


class Place10(Place):
    def __init__(self, bet_amount):
        self.name = "Place10"
        self.winning_numbers = [10]
        self.losing_numbers = [7]
        self.payoutratio = 9 / 5
        super().__init__(bet_amount)


"""
Field bet
"""


class Field(Bet):
    """
    Parameters
    ----------
    double : list
        Set of numbers that pay double on the field bet (default = [2,12])
    triple : list
        Set of numbers that pay triple on the field bet (default = [])
    """

    def __init__(self, bet_amount, double=[2, 12], triple=[]):
        self.name = "Field"
        self.double_winning_numbers = double
        self.triple_winning_numbers = triple
        self.winning_numbers = [2, 3, 4, 9, 10, 11, 12]
        self.losing_numbers = [5, 6, 7, 8]
        super().__init__(bet_amount)

    def _update_bet(self, table_object, dice_object):
        status = None
        win_amount = 0

        if dice_object.total in self.triple_winning_numbers:
            status = "win"
            win_amount = 3 * self.bet_amount
        elif dice_object.total in self.double_winning_numbers:
            status = "win"
            win_amount = 2 * self.bet_amount
        elif dice_object.total in self.winning_numbers:
            status = "win"
            win_amount = 1 * self.bet_amount
        elif dice_object.total in self.losing_numbers:
            status = "lose"

        return status, win_amount


"""
Don't pass and Don't come bets
"""


class DontPass(Bet):
    # TODO: make this require that table_object.point = "Off",
    #  probably better in the player module
    def __init__(self, bet_amount):
        self.name = "DontPass"
        self.winning_numbers = [2, 3]
        self.losing_numbers = [7, 11]
        self.push_numbers = [12]
        self.payoutratio = 1.0
        self.prepoint = True
        super().__init__(bet_amount)

    def _update_bet(self, table_object, dice_object):
        status = None
        win_amount = 0

        if dice_object.total in self.winning_numbers:
            status = "win"
            win_amount = self.payoutratio * self.bet_amount
        elif dice_object.total in self.losing_numbers:
            status = "lose"
        elif dice_object.total in self.push_numbers:
            status = "push"
        elif self.prepoint:
            self.winning_numbers = [7]
            self.losing_numbers = [dice_object.total]
            self.push_numbers = []
            self.prepoint = False

        return status, win_amount


"""
Don't pass/Don't come lay odds
"""


class LayOdds(Bet):
    def __init__(self, bet_amount, bet_object):
        super().__init__(bet_amount)
        self.name = "LayOdds"
        self.subname = "".join(str(e) for e in bet_object.losing_numbers)
        self.winning_numbers = bet_object.winning_numbers
        self.losing_numbers = bet_object.losing_numbers

        if self.losing_numbers == [4] or self.losing_numbers == [10]:
            self.payoutratio = 1 / 2
        elif self.losing_numbers == [5] or self.losing_numbers == [9]:
            self.payoutratio = 2 / 3
        elif self.losing_numbers == [6] or self.losing_numbers == [8]:
            self.payoutratio = 5 / 6



"""
Various betting strategies that are based on conditions of the CrapsTable.
Each strategy must take a table and a player_object, and implicitly 
uses the methods from the player object.
"""

"""
Fundamental Strategies
"""


def passline(player, table, unit=5, strat_info=None):
    # Pass line bet
    if table.point == "Off" and not player.has_bet("PassLine"):
        player.bet(PassLine(unit))


def passline_odds(player, table, unit=5, strat_info=None, mult=1):
    passline(player, table, unit)
    # Pass line odds
    if mult == "345":
        if table.point == "On":
            if table.point.number in [4, 10]:
                mult = 3
            elif table.point.number in [5, 9]:
                mult = 4
            elif table.point.number in [6, 8]:
                mult = 5
    else:
        mult = float(mult)

    if (
        table.point == "On"
        and player.has_bet("PassLine")
        and not player.has_bet("Odds")
    ):
        player.bet(Odds(mult * unit, player.get_bet("PassLine")))


def passline_odds2(player, table, unit=5, strat_info=None):
    passline_odds(player, table, unit, strat_info=None, mult=2)


def passline_odds345(player, table, unit=5, strat_info=None):
    passline_odds(player, table, unit, strat_info=None, mult="345")


def pass2come(player, table, unit=5, strat_info=None):
    passline(player, table, unit)

    # Come bet (2)
    if table.point == "On" and player.num_bet("Come") < 2:
        player.bet(Come(unit))


def place(player, table, unit=5, strat_info={"numbers": {6, 8}}, skip_point=True):
    strat_info["numbers"] = set(strat_info["numbers"]).intersection({4, 5, 6, 8, 9, 10})
    if skip_point:
        strat_info["numbers"] -= {table.point.number}

    # Place the provided numbers when point is ON
    if table.point == "On":
        if not player.has_bet("Place4") and 4 in strat_info["numbers"]:
            player.bet(Place4(unit))
        if not player.has_bet("Place5") and 5 in strat_info["numbers"]:
            player.bet(Place5(unit))
        if not player.has_bet("Place6") and 6 in strat_info["numbers"]:
            player.bet(Place6(6 / 5 * unit))
        if not player.has_bet("Place8") and 8 in strat_info["numbers"]:
            player.bet(Place8(6 / 5 * unit))
        if not player.has_bet("Place9") and 9 in strat_info["numbers"]:
            player.bet(Place9(unit))
        if not player.has_bet("Place10") and 10 in strat_info["numbers"]:
            player.bet(Place10(unit))

    # Move the bets off the point number if it shows up later
    if skip_point and table.point == "On":
        if player.has_bet("Place4") and table.point.number == 4:
            player.remove(player.get_bet("Place4"))
        if player.has_bet("Place5") and table.point.number == 5:
            player.remove(player.get_bet("Place5"))
        if player.has_bet("Place6") and table.point.number == 6:
            player.remove(player.get_bet("Place6"))
        if player.has_bet("Place8") and table.point.number == 8:
            player.remove(player.get_bet("Place8"))
        if player.has_bet("Place9") and table.point.number == 9:
            player.remove(player.get_bet("Place9"))
        if player.has_bet("Place10") and table.point.number == 10:
            player.remove(player.get_bet("Place10"))


def place68(player, table, unit=5, strat_info=None):
    passline(player, table, unit, strat_info=None)
    # Place 6 and 8 when point is ON
    p_has_place_bets = player.has_bet(
        "Place4", "Place5", "Place6", "Place8", "Place9", "Place10"
    )
    if table.point == "On" and not p_has_place_bets:
        if table.point.number == 6:
            player.bet(Place8(6 / 5 * unit))
        elif table.point.number == 8:
            player.bet(Place6(6 / 5 * unit))
        else:
            player.bet(Place8(6 / 5 * unit))
            player.bet(Place6(6 / 5 * unit))


def dontpass(player, table, unit=5, strat_info=None):
    # Don't pass bet
    if table.point == "Off" and not player.has_bet("DontPass"):
        player.bet(DontPass(unit))


def layodds(player, table, unit=5, strat_info=None, win_mult=1):
    # Assume that someone tries to win the `win_mult` times the unit on each bet, which corresponds
    # well to the max_odds on a table.
    # For `win_mult` = "345", this assumes max of 3-4-5x odds
    dontpass(player, table, unit)

    # Lay odds for don't pass
    if win_mult == "345":
        mult = 6.0
    else:
        win_mult = float(win_mult)
        if table.point == "On":
            if table.point.number in [4, 10]:
                mult = 2 * win_mult
            elif table.point.number in [5, 9]:
                mult = 3 / 2 * win_mult
            elif table.point.number in [6, 8]:
                mult = 6 / 5 * win_mult

    if (
        table.point == "On"
        and player.has_bet("DontPass")
        and not player.has_bet("LayOdds")
    ):
        player.bet(LayOdds(mult * unit, player.get_bet("DontPass")))


class Table(object):
    
    def __init__(self):
        self.players = []
        self.player_has_bets = False
        # TODO: I think strat_info should be attached to each player object
        self.strat_info = {}
        self.point = _Point()
        self.dice = Dice()
        self.bet_update_info = None
        self.payouts = {"fielddouble": [2, 12], "fieldtriple": []}
        self.pass_rolls = 0
        self.last_roll = None
        self.n_shooters = 1

    @classmethod
    def with_payouts(cls, **kwagrs):
        table = cls()
        for name, value in kwagrs.items():
            table.payouts[name] = value
        return table

    def set_payouts(self, name, value):
        self.payouts[name] = value

    def add_player(self, player_object):
        """ Add player object to the table """
        if player_object not in self.players:
            self.players.append(player_object)
            self.strat_info[player_object] = None

    def run(self, max_rolls, max_shooter=float("inf"), verbose=True, runout=False):
        """
        Runs the craps table until a stopping condition is met.
        Parameters
        ----------
        max_rolls : int
            Maximum number of rolls to run for
        verbose : bool
            If true, print results from table during each roll
        runout : bool
            If true, continue past max_rolls until player has no more bets on the table
        """
        self.dice = Dice()
        if verbose:
            print("Welcome to the Craps Table!")

        # make sure at least one player is at table
        if not self.players:
            self.add_player(Player(500, "Player1"))
        if verbose:
            print(f"Initial players: {[p.name for p in self.players]}")

        # maybe wrap this into update table or something
        self.total_player_cash = sum(
            [p.total_bet_amount + p.bankroll for p in self.players]
        )

        continue_rolling = True
        while continue_rolling:

            # players make their bets
            self._add_player_bets()
            for p in self.players:
                bets = [
                    f"{b.name}{b.subname}, ${b.bet_amount}" for b in p.bets_on_table
                ]
                if verbose:
                    print(f"{p.name}'s current bets: {bets}")

            self.dice.roll()
            self._update_player_bets(self.dice, verbose)
            self._update_table(self.dice)
            if verbose:
                print("")
                print("Dice out!")
                print(f"Shooter rolled {self.dice.total} {self.dice.result}")
                print(f"Point is {self.point.status} ({self.point.number})")
                print(f"Total Player Cash is ${self.total_player_cash}")

            # evaluate the stopping condition
            if runout:
                continue_rolling = (
                    self.dice.n_rolls < max_rolls
                    and self.n_shooters <= max_shooter
                    and self.total_player_cash > 0
                ) or self.player_has_bets
            else:
                continue_rolling = (
                    self.dice.n_rolls < max_rolls
                    and self.n_shooters <= max_shooter
                    and self.total_player_cash > 0
                )

    def _add_player_bets(self):
        """ Implement each player's betting strategy """
        """ TODO: restrict bets that shouldn't be possible based on table"""
        """ TODO: Make the unit parameter specific to each player, and make it more general """
        for p in self.players:
            self.strat_info[p] = p._add_strategy_bets(
                self, unit=5, strat_info=self.strat_info[p]
            )  # unit = 10 to change unit
            # TODO: add player.strat_kwargs as optional parameter (currently manually changed in CrapsTable)

    def _update_player_bets(self, dice, verbose=False):
        """ check bets for wins/losses, payout wins to their bankroll, remove bets that have resolved """
        self.bet_update_info = {}
        for p in self.players:
            info = p._update_bet(self, dice, verbose)
            self.bet_update_info[p] = info

    def _update_table(self, dice):
        """ update table attributes based on previous dice roll """
        self.pass_rolls += 1
        if self.point == "On" and dice.total == 7:
            self.n_shooters += 1
        if self.point == "On" and (dice.total == 7 or dice.total == self.point.number):
            self.pass_rolls = 0

        self.point.update(self.dice)
        self.total_player_cash = sum(
            [p.total_bet_amount + p.bankroll for p in self.players]
        )
        self.player_has_bets = sum([len(p.bets_on_table) for p in self.players]) >= 1
        self.last_roll = dice.total

    def _get_player(self, player_name):
        [p for p in self.players if p.name == player_name]
        for p in self.players:
            if p.name == player_name:
                return p
        return False


class _Point(object):
    """
    The point on a craps table.
    Parameters
    ----------
    NONE
    Attributes
    ----------
    status : str
        Either 'On' or 'Off', depending on whether a point is set
    number : int
        The point number (in [4, 5, 6, 8, 9, 10]) is status == 'On'
    """

    def __init__(self):
        self.status = "Off"
        self.number = None

    def __eq__(self, other):
        return self.status == other

    def update(self, dice_object: Dice):
        if self.status == "Off" and dice_object.total in [4, 5, 6, 8, 9, 10]:
            self.status = "On"
            self.number = dice_object.total
        elif self.status == "On" and dice_object.total in [7, self.number]:
            self.status = "Off"
            self.number = None


if __name__ == "__main__":
    import sys 

def main():
  table = Table()
  strategy_variances = [passline, passline_odds, passline_odds2,  passline_odds345, pass2come, place, place68, dontpass, layodds]
  name = str(input("Enter your name: "))
  bankroll = int(input("Enter the starting amount of cash: "))
  your_strat = int(input('Choose the strategy for the game (passline(1), passline_odds(2), passline_odds2(3),  passline_odds345(4), pass2come(5), place(6), place68(7), dontpass(8), layodds(9))'))
  your_strat1 = strategy_variances[your_strat]
  max_rolls = int (input("Enter the maximum number of rolls: ") )
  player_ = Player(name = name, bankroll = bankroll, bet_strategy = your_strat1)
  table.add_player(player_)
  table.run(max_rolls = max_rolls)
main()



Enter your name: ss
Enter the starting amount of cash: 120
Choose the strategy for the game (passline(1), passline_odds(2), passline_odds2(3),  passline_odds345(4), pass2come(5), place(6), place68(7), dontpass(8), layodds(9))3
Enter the maximum number of rolls: 10
Welcome to the Craps Table!
Initial players: ['ss']
ss's current bets: ['PassLine, $5.0']
ss won $5.0 on PassLine bet!

Dice out!
Shooter rolled 7 [6 1]
Point is Off (None)
Total Player Cash is $125.0
ss's current bets: ['PassLine, $5.0']

Dice out!
Shooter rolled 6 [4 2]
Point is On (6)
Total Player Cash is $125.0
ss's current bets: ['PassLine, $5.0', 'Odds6, $25.0']

Dice out!
Shooter rolled 8 [4 4]
Point is On (6)
Total Player Cash is $125.0
ss's current bets: ['PassLine, $5.0', 'Odds6, $25.0']
ss won $5.0 on PassLine bet!
ss won $30.0 on Odds bet!

Dice out!
Shooter rolled 6 [3 3]
Point is Off (None)
Total Player Cash is $160.0
ss's current bets: ['PassLine, $5.0']

Dice out!
Shooter rolled 5 [4 1]
Point is On (5)
Total P